#### Initializing Dims and Facts (once only)
* Create empty tables
* Create dims, facts stored procedures
* Execute dims, facts stored procedures

#### Updating Dims and Facts with new data
* Run R script to prep data for staging area ([elmer_prep_faa_enplanements.R](https://github.com/psrc/trends-airports/blob/master/elmer/elmer_prep_faa_enplanements.R))
* Execute dims stored procedures
* Execute facts stored procedures


In [9]:
SELECT top 10 *
FROM Sandbox.Christy.faa_enplanements;

In [1]:
SELECT DISTINCT [year]
FROM Sandbox.Christy.faa_enplanements
ORDER BY [year];

(12 rows affected)

Total execution time: 00:00:00.212

year
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015


In [5]:
USE Elmer
SELECT ad.*, ef.*
FROM faa.airport_dims as ad
JOIN faa.enplanement_facts as ef
ON ad.airport_dim_id = ef.airport_dim_id
WHERE city_name = 'Atlanta';

# Dimensions

#### Alter dimensions stored procedure

In [0]:
ALTER PROCEDURE [faa].[merge_enplanements_dim] 
as

;with cte as (
	select ISNULL(RO, '--') AS RO, 
		ISNULL(ST, '--') AS ST, 
		ISNULL(Locid, '--') as Locid, 
		ISNULL(City, '--') as City, 
		ISNULL(Airportname, '--') as Airportname, 
		ISNULL(SL, '--') as SL, 
		ISNULL(Hub, '--') as Hub
	--from Sandbox.Christy.faa_enplanements
    from stg.faa_enplanements
	group by RO, ST, Locid, City, Airportname, SL, Hub
)
MERGE  faa.airport_dims as target
USING cte as source
ON (
	target.ro = source.RO
	and target.st = source.ST
	and target.loc_id = source.Locid
	and target.city_name = source.City
	and target.airport_name = source.Airportname
	and target.sl = source.SL
	and target.hub = source.Hub
)
WHEN NOT MATCHED THEN 
INSERT (ro, st, loc_id, city_name, airport_name, sl, hub)
VALUES (source.RO, 
	source.ST,
	source.Locid,
	source.City,
	source.Airportname,
	source.SL,
	source.Hub
);

Create/update dim table

In [0]:
EXEC faa.merge_enplanements_dim

Check dim table

In [0]:
SELECT *
From Elmer.faa.airport_dims;

# Facts

In [4]:
SELECT *
FROM Elmer.faa.enplanement_facts;

Test join before creating stored procedure

In [15]:
SELECT *
from Christy.faa_enplanements as fe
        INNER JOIN Elmer.faa.airport_dims as ad
        ON fe.RO = ad.ro
        and fe.ST = ad.st
        and fe.Locid = ad.loc_id
        and fe.City = ad.city_name
        and fe.Airportname = ad.airport_name
        and fe.SL = ad.sl
        and fe.Hub = ad.hub

#### Alter a stored procedure for merging FAA Enplanements Facts

In [13]:
ALTER PROCEDURE [faa].[merge_enplanements_fact] 
as

;with cte as (
	select ad.airport_dim_id, fe.enplanements, fe.year
    from stg.faa_enplanements as fe 
	--from Sandbox.Christy.faa_enplanements as fe
        INNER JOIN Elmer.faa.airport_dims as ad
        ON fe.RO = ad.ro
        and fe.ST = ad.st
        and fe.Locid = ad.loc_id
        and fe.City = ad.city_name
        and fe.Airportname = ad.airport_name
        and ISNULL(fe.SL, '--') = ad.sl
        and ISNULL(fe.Hub, '--') = ad.hub
)
MERGE faa.enplanement_facts as target
USING cte as source
ON (target.airport_dim_id = source.airport_dim_id
	and target.enplanements = source.enplanements
	and target.data_year = source.year
)
WHEN NOT MATCHED THEN 
INSERT (airport_dim_id, data_year, enplanements)
VALUES (source.airport_dim_id,
    source.year, 
	source.enplanements
);

Commands completed successfully.

Total execution time: 00:00:00.013

Create/update the Fact table

In [14]:
EXEC faa.merge_enplanements_fact

(518 rows affected)

Total execution time: 00:00:00.056

Check fact table

In [15]:
SELECT *
FROM faa.enplanement_facts;

In [5]:
USE Elmer

Commands completed successfully.

Total execution time: 00:00:00.010